In [6]:
import json
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import unicodecsv as csv
import pandas as pd
import numpy as np
import warnings
from vader import *
warnings.filterwarnings('ignore')

In [17]:
def twitter_stream(file_name, coordinates, tweets_limit):
    '''
       Parameters
       ----------
       file_name          : string, file's name to be saved as.
       coordinates        : list, box corners' coordinates to catch tweets.
       tweets_limit       : int, number of tweets to capture.

       Returns
       ----------
       file with captured tweets
    '''
    file =  open(file_name+'.txt', 'a+', encoding='utf-8')
    
    ckey = "***"
    csecret = "***"
    atoken = "***"
    asecret = "***"
    
    class listener(StreamListener):

        def __init__(self, api=None):
            super(listener, self).__init__()
            self.num_tweets = 0

        def on_data(self, data):
            # Twitter returns data in JSON format - we need to decode it first
            try:
                decoded = json.loads(data)
            except Exception as e:
                print (e) #we don't want the listener to stop
                return True

            if decoded.get('geo') is not None:
                location = decoded.get('geo').get('coordinates')
            elif decoded.get('user').get('location') != None:
                location = '[' + decoded.get('user').get('location') + ']'
            else:
                location = decoded.get('user').get('location')
            text = decoded['text'].replace('\n',' ')
            user = '@' + decoded.get('user').get('screen_name')
            created = decoded.get('created_at')

            file.write('{0}|{1}|{2}|{3}\n'.format(user,location,created,text))
                
            print('{0}|{1}|{2}|{3}|{4}\n'.format(self.num_tweets,user,location,created,text))

            self.num_tweets += 1
            if self.num_tweets < tweets_limit:
                return True
            else:
                file.close()
                return False

        def on_error(self, status):
            print(status)

    if __name__ == '__main__':
        print('Starting')

        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        twitterStream = Stream(auth, listener())
        twitterStream.filter(locations=coordinates)

    
def get_coordinates(file_name, coordinates, vader_score = False, fill_random = True):
    '''
       Parameters
       ----------
       file_name          : string, file's name to be analized.
       coordinates        : list, box corners' coordinates.
       vader_score        : bool, default False, gets sentiment score of file's tweets and saves coordinates' list
                            grouped by sentiment score (positive, neutral and negative) in three different files.
                            If False, it will only save one file with all the coordinates.
       fill_random        : bool, default True, fills empty coordinates with random numbers within coordinates range.

       Returns
       ----------
       file with coordinates (three files if vader_score = True). 
    '''
    lon_min, lat_min, lon_max, lat_max = coordinates
    tweets = pd.read_table(file_name + '.txt', sep = "|", names = ["user","location","date","text"])
    
    if fill_random:
        tweets['lat'] = tweets.location.apply(lambda x: x.split(',')[0].replace('[','') if ',' in x else np.nan)
        tweets['lon'] = tweets.location.apply(lambda x: x.split(',')[1].replace(']','') if ',' in x else np.nan)

        tweets['lat'] = tweets.lat.convert_objects(convert_numeric=True)
        tweets['lon'] = tweets.lon.convert_objects(convert_numeric=True)
        tweets.loc[tweets['lat'].isnull(), 'lat'] = np.random.uniform(lat_min, lat_max, tweets.loc[tweets['lat'].isnull(), 'lat'].size)
        tweets.loc[tweets['lon'].isnull(), 'lon'] = np.random.uniform(lon_min, lon_max, tweets.loc[tweets['lon'].isnull(), 'lon'].size)
    else:
        tweets['lat'] = tweets.location.apply(lambda x: x.split(',')[0].replace('[','') if ',' in x else None)
        tweets['lon'] = tweets.location.apply(lambda x: x.split(',')[1].replace(']','') if ',' in x else None)

        tweets['lat'] = tweets.lat.convert_objects(convert_numeric=True)
        tweets['lon'] = tweets.lon.convert_objects(convert_numeric=True)
        
        tweets = tweets.dropna()
        
    if vader_score:
        analyzer = SentimentIntensityAnalyzer()
        tweets['score'] = tweets.text.apply(lambda x: analyzer.polarity_scores(x)['compound'])
        def WriteCSV(csv_file, data_list):
            with open(csv_file, 'wb') as csvfile:
                writer = csv.writer(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
                for data in data_list:
                    writer.writerow(data) 
            return

        data_list_pos = zip(tweets.loc[tweets['score'] > 0.5, 'lat'], tweets.loc[tweets['score'] > 0.5, 'lon'])
        data_list_neg = zip(tweets.loc[tweets['score'] < -0.5, 'lat'], tweets.loc[tweets['score'] <- 0.5, 'lon'])
        data_list_neu = zip(tweets.loc[(tweets['score'] > -0.5) & (tweets['score'] < 0.5), 'lat'],\
                            (tweets.loc[(tweets['score'] > -0.5) & (tweets['score'] < 0.5), 'lon']))

        WriteCSV(file_name + '_pos_points.csv', data_list_pos)
        WriteCSV(file_name + '_neg_points.csv', data_list_neg)
        WriteCSV(file_name + '_neu_points.csv', data_list_neu)
    else:
        data_list = zip(tweets['lat'],tweets['lon'])
        csv_file = file_name + '_points.csv'

        with open(csv_file, 'wb') as csvfile:
            writer = csv.writer(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
            for data in data_list:
                writer.writerow(data)
        
        
def download_map(file_name, coordinates, quality = 15):
    '''
       Parameters
       ----------
       file_name          : string, file's name to be saved as.
       coordinates        : list, box corners' coordinates.
       quality            : int, default 15, map quality level.

       Returns
       ----------
       image file of the map.
       
       Note: Requires to install osmviz library to work: pip install osmviz
    '''
    from osmviz.manager import PILImageManager, OSMManager
    import PIL.Image as Image
    
    lon_min, lat_min, lon_max, lat_max = coordinates

    imgr = PILImageManager('RGB')
    osm = OSMManager(image_manager=imgr)
    image, bnds = osm.createOSMImage((lat_min, lat_max, lon_min, lon_max), quality)
    wh_ratio = float(image.size[0]) / image.size[1]
    image2 = image.resize((int(800*wh_ratio), 800), Image.ANTIALIAS)
    del image
    image2.save(file_name + '_map.bmp')

In [8]:
coordinates = [-103.466492,20.522538,-103.213806,20.743593]

file_name = 'tweets_try7'

In [ ]:
twitter_stream(file_name, coordinates, 20)

In [18]:
get_coordinates(file_name, coordinates, vader_score = False, fill_random = True)

In [ ]:
download_map(file_name, coordinates, 14)

In [12]:
#Navy Blue
%run heatmap.py -I file_name_map.bmp -o tweets_try_vader1.png -m 1996080ff -M 1996080ff\
    -e 20.522538,-103.466492,20.743593,-103.213806 tweets_try7_neu_points.csv

In [15]:
#Green
%run heatmap.py -I tweets_try_vader1.png -o tweets_try_vader1.png -m 359ffffff -M 359ffffff\
    -e 20.522538,-103.466492,20.743593,-103.213806 tweets_try7_neu_points.csv

In [18]:
#Yellow
%run heatmap.py -I tweets_try_vader1.png -o tweets_try_vader1.png -m 130ffffff -M 130ffffff\
    -e 20.522538,-103.466492,20.743593,-103.213806 tweets_try7_neu_points.csv

In [28]:
#Red
%run heatmap.py -I tweets_try_vader1.png -o tweets_try_vader1.png -m 100ffff00 -M 100ffff00\
    -e 20.522538,-103.466492,20.743593,-103.213806 -r 30 tweets_try7_neu_points.csv